In [1]:
from pymorphy2 import MorphAnalyzer
from gensim.models import KeyedVectors
from time import time
import numpy as np
import pandas as pd

load model

In [2]:
model = KeyedVectors.load_word2vec_format('word_to_vec.bin', binary=True)

In [3]:
morph = MorphAnalyzer()
def handle_word(word):
    word = word.lower().strip('(.,!?:;\'"«»—-–)')
    p = morph.parse(word)[0]
    return p.normal_form
def get_average_vector(words):
    vectors = []
    for word in words:
        parsed = handle_word(word)
        if parsed in model:
            vectors.append(model[parsed])
        else:
            vectors.append(np.zeros(100))
    return np.mean(vectors, axis=0)


print(get_average_vector(['привет', 'как', 'дела']))

[ 0.01319534 -0.44012567 -0.24828333 -0.30982634  0.44310102  0.18418433
  0.12236699 -0.06977067 -0.22141068  0.33174     0.04571233  0.4523003
  0.05751466 -0.25541267 -0.10298267  0.32320166  0.28154334  0.05418434
  0.45345068  0.20138998 -0.389215    0.384002    0.34564066 -0.25499168
  0.16727601  0.03322067  0.00858967  0.066601   -0.05029468 -0.19079167
  0.04519533 -0.027845   -0.257506    0.01388633 -0.04997567 -0.03241967
  0.014428   -0.11890599  0.029228   -0.34692034 -0.26247802  0.298645
  0.3107127  -0.14327334 -0.15045999  0.129627    0.20942901 -0.05834166
  0.17391133 -0.16441034  0.12307634 -0.14922099 -0.061433    0.22274001
  0.13244967  0.02664267  0.07971966  0.3478087  -0.11845666 -0.29305732
 -0.074766   -0.02340234 -0.02104733 -0.14968766  0.07986266  0.24631031
  0.020461   -0.23431699  0.6417493  -0.041198    0.48819467  0.04603934
  0.02268433 -0.2872503  -0.005793   -0.10669366 -0.32312167  0.13104033
  0.01454767 -0.22922999  0.085271   -0.13496734  0.05

#### check model 

In [4]:
# all_words = list(set([word for text in data['uu_usl_name'].values[:10000] for word in text.split()]))
# print(len([handle_word(word) for word in all_words if handle_word(word) in model]))

In [3]:
import pandas as pd
data = pd.read_csv('dataset.csv', sep=',')
data

,uu_usl_name,CLASS_ID,SERVICE_CLASS_CONFIRMED,CNT_DIST_USL_BY_CLASS,CNT_NUM_USL
0,АМБУЛАТОРНАЯ ПОМОЩЬ,1000,NaN,12614,812414
1,АМБУЛАТОРНАЯ ПОМОЩЬ,1292,NaN,777,812414
2,КОНСУЛЬТАТИВНО-ДИАГНОСТИЧЕСКАЯ ПОМОЩЬ,1000,NaN,12614,740580
3,Сбербанк Премьер,3200,1.0,3974,489254
4,Сбербанк Премьер,8350,1.0,104,489254
...,...,...,...,...,...
81601,Прием врачом-онкологом первичный амбулаторный,9000,NaN,169,1
81602,Проводниковая анестезия,9000,NaN,169,1
81603,Проводниковая анестезия (блокада по Оберсту-Лу...,9000,NaN,169,1
81604,Проводниковая анестезия Sol. Ultracani D.S,9000,NaN,169,1


In [6]:
#clear data
columns_to_delete = ['SERVICE_CLASS_CONFIRMED', 'CNT_DIST_USL_BY_CLASS', 'CNT_NUM_USL']
data = data.drop(columns_to_delete, axis=1)
data = data[data.CLASS_ID != 9000]

In [7]:
pre_X = [get_average_vector(text.split()) for text in data['uu_usl_name'].values]

In [8]:
del model

In [9]:

X = pd.DataFrame(pre_X, dtype=float).fillna(0)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.191372,-0.244398,-0.168261,0.232595,0.652055,0.053336,0.187999,-0.201186,-0.614163,0.251317,...,0.573032,0.079496,0.545428,0.539856,0.176765,0.349771,0.218045,0.271989,0.489833,-0.212393
1,-0.191372,-0.244398,-0.168261,0.232595,0.652055,0.053336,0.187999,-0.201186,-0.614163,0.251317,...,0.573032,0.079496,0.545428,0.539856,0.176765,0.349771,0.218045,0.271989,0.489833,-0.212393
2,-0.077774,-0.288235,0.035577,0.285684,0.560462,-0.148274,0.155463,-0.072728,-0.568736,0.051853,...,0.491524,-0.014641,0.279054,0.690858,0.183297,0.060691,0.288101,0.156691,0.311036,-0.134095
3,-0.372827,-0.135439,0.156183,-0.010491,0.134077,-0.344258,-0.224037,0.029237,-0.082438,0.520093,...,-0.086702,0.361299,-0.205426,0.239095,-0.405685,0.158046,-0.211746,-0.081887,-0.014821,0.552056
4,-0.372827,-0.135439,0.156183,-0.010491,0.134077,-0.344258,-0.224037,0.029237,-0.082438,0.520093,...,-0.086702,0.361299,-0.205426,0.239095,-0.405685,0.158046,-0.211746,-0.081887,-0.014821,0.552056


In [10]:
y = data['CLASS_ID'].values

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

clf = RandomForestClassifier(
    n_estimators=20,
    max_depth=200,
    random_state=0,
    n_jobs=-1,
    min_samples_split=10,
    min_samples_leaf=10,
)



In [12]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=200, min_samples_leaf=10, min_samples_split=10,
                       n_estimators=20, n_jobs=-1, random_state=0)

In [1]:
pred_X = clf.predict(X_test)
accuracy_score(y_test, pred_X)

NameError: name 'clf' is not defined

'hello'